In [1]:
import build_network__
net = build_network__.BuildModel(1,1,5) # not statuc can be reset

Alpha Griffin TF_Curses Project


In [2]:
import tensorflow as tf
import os, sys, datetime

In [3]:
# you need to know your dataset sizes to make the placeholders
dataset_h = 66
dataset_w = 200
dataset_c = 3 
dataset_classes = 5

In [4]:
 # get common stuff done
input_tensor, \
label_tensor, \
learn_rate, \
keep_prob = net.build_inputs(dataset_h, 
                             dataset_w, 
                             dataset_c, 
                             dataset_classes
                             )
#print("# Built Input {}".format(index))

In [ ]:
thing1 = tf.placeholder(tf.float32, [None,
                                             dataset_h,
                                             dataset_w,
                                             dataset)
tf.add_to_collection("thing1", thing2)

In [ ]:
with tf.variable_scope("inputs"):
    thing2 = tf.placeholder(tf.float32, [None, 
                                                     dataset_h, 
                                                     dataset_w,
                                                     dataset_c])
tf.add_to_collection("input_tensor", thing2)

In [ ]:
print(thing2)

In [ ]:
print(thing1)

In [5]:
# take the inputed variables and implement them
x_image = net.build_outputs(input_tensor,
                            5,
                            5,
                            5
                            #keep_prob
                           )

In [6]:
print(keep_prob)

Tensor("inputs/Keep_prob:0", dtype=float32)


In [7]:
print(x_image)

Tensor("FC_layers/add_4:0", shape=(?, 5), dtype=float32)


In [8]:
print(input_tensor)

Tensor("inputs/Placeholder:0", shape=(?, 66, 200, 3), dtype=float32)


In [9]:
### TRAINING METHOD FROM MUPEN #### MEPHOD?
with tf.variable_scope("mupen_method"):
    cost = tf.reduce_mean(tf.square(tf.subtract(input_tensor, x_image)))
    train_vars = tf.trainable_variables()
    loss = cost +\
			tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) *\
			0.001
    # loss = cost + training_vars
tf.add_to_collection("loss", loss)
tf.summary.scalar("train_cost", loss)

ValueError: Dimensions must be equal, but are 3 and 5 for 'mupen_method/Sub' (op: 'Sub') with input shapes: [?,66,200,3], [?,5].

In [ ]:
train, loss, softmax = net.training_method(x_image, input_tensor, learn_rate)

In [ ]:
job_name = "AlphaGriffin_TF_Server"
known_machines = []
# this is another process... worker...
known_machines.append('localhost:2222')
# this is this process
known_machines.append('localhost:2223')
#known_machines.append( {'cool_name': ['sock_ip:port', 'localhost:2223']} )
known_machines_dict = { '{}'.format(job_name): [x for x in known_machines] }
len_machines = len(known_machines)
print("There are {} known machines".format(len_machines))
print(known_machines_dict)

In [ ]:
cluster = tf.train.ClusterSpec(known_machines_dict)

In [ ]:
# this is the HOME position... "you are here"
server = tf.train.Server(cluster,
                         job_name,
                         task_index=1) 
print(server)

In [ ]:
# for each worker/server/w/e in the KNOWN CLUSTER... so job name would be known here.
#index = 0
def build_graphs(num_conv, num_fc, num_outputs):
    all_graphs = []
    for index in range(len_machines):
        current_context = tf.Graph()
        current_worker = "/job:{}/task:{}".format(job_name, index)
        print("With {} build graph {}".format(current_worker, index))
        with tf.device(tf.train.replica_device_setter(worker_device=current_worker,
                                                      cluster=cluster)):
            # get common stuff done
            input_tensor, \
            label_tensor, \
            learn_rate, \
            keep_prob = net.build_inputs(dataset_h, 
                                         dataset_w, 
                                         dataset_c, 
                                         dataset_classes
                                         )
            print("# Built Input {}".format(index))
            
            # take the inputed variables and implement them
            x_image = net.build_outputs(input_tensor,
                                        num_conv,
                                        num_fc,
                                        num_outputs,
                                        keep_prob)
            
            print("# Built Output {}".format(index))
            
            # build training ops
            train_op, loss, softmax = net.training_method(x_image, 
                                                          input_tensor,
                                                          learn_rate
                                                          )
            # Finished Building Graph            
        all_graphs.append(current_context)
    return all_graphs

In [ ]:
mastergraph = build_graphs(5,5,5)
print(mastergraph)